In [1]:
import requests

In [2]:
base_url = 'https://blockstream.info/api'

In [20]:
current_point = 855543
r = requests.get(f"{base_url}/blocks/{current_point}")

In [21]:
rows = [(x['height'], x['timestamp']) for x in r.json()]

In [22]:
rows[0]

(855543, 1722881915)

In [1]:
import sqlite3
import time

In [31]:
conn = sqlite3.connect('block_heights.db')
cur = conn.cursor()

In [11]:
# Create a table (if it doesn't exist)
cur.execute('''CREATE TABLE IF NOT EXISTS block_times (
    block_height INTEGER PRIMARY KEY,
    timestamp INTEGER NOT NULL
);''')

conn.commit()

In [ ]:
current_point = None

In [34]:
current_point

849143

In [ ]:
while True:
    try:
        r = requests.get(f"{base_url}/blocks/{current_point-1}")
        r.raise_for_status()        
        rows = [(x['height'], x['timestamp']) for x in r.json()]
        cur.executemany("INSERT INTO block_times (block_height, timestamp) VALUES (?, ?)", rows)
        conn.commit()

        current_point = rows[-1][0]
        print(f'{current_point} done')
    
    except requests.exceptions.HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  # HTTP error occurred
        time.sleep(0.3)
        print('retrying...')
        continue

    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
        break

## Did we miss any block?

In [38]:
cur.execute(
"""
    SELECT COUNT(*) AS rows,
    MAX(block_height) - MIN(block_height) + 1
    FROM block_times
""").fetchall()

[(7490, 7490)]

# Make the best block_heights table

In [2]:
import pandas as pd

In [3]:
def read_from_sqlite(path):
    sqlite_conn = sqlite3.connect(path)
    df = pd.read_sql_query("SELECT * FROM block_times", sqlite_conn)
    sqlite_conn.close()
    return df

In [4]:
block_times_1 = read_from_sqlite('block_heights.db')

In [5]:
block_times_2 = read_from_sqlite('/mnt/e/Data/fedi/fedimint-observer.db')

In [6]:
block_times_3 = read_from_sqlite('bitcoin_principles.db')

In [7]:
block_times_4 = read_from_sqlite('fedimint-observer/fedimint_observer.db')

In [8]:
len(block_times_1), len(block_times_2), len(block_times_3), len(block_times_4)

(7490, 34721, 29362, 5646)

In [9]:
display(block_times_1.agg(['min','max']))
display(block_times_2.agg(['min','max']))
display(block_times_3.agg(['min','max']))
display(block_times_4.agg(['min','max']))

,block_height,timestamp
min,848063,1718476789
max,855552,1722887754


,block_height,timestamp
min,820001,1701860884
max,854721,1722376743


,block_height,timestamp
min,820001,1701860884
max,849362,1719267429


,block_height,timestamp
min,850001,1719689806
max,855646,1722946456


In [10]:
block_times_full = pd.concat([block_times_1, block_times_2, block_times_3, block_times_4]).drop_duplicates()

In [11]:
block_times_full.agg(['min','max'])

,block_height,timestamp
min,820001,1701860884
max,855646,1722946456


In [12]:
len(block_times_full)

35646

In [13]:
block_times_full.groupby('block_height').timestamp.count().reset_index().groupby('timestamp').count()

,block_height
timestamp,
1,35646


## Replacing table in duckdb

In [14]:
import duckdb

In [15]:
duck_conn = duckdb.connect('freedom_one-duckdb.db')
duck_conn.register('temp_df', block_times_full)
duck_conn.execute("CREATE OR REPLACE TABLE block_times AS SELECT * FROM temp_df")
duck_conn.unregister('temp_df')
duck_conn.close()